In [1]:
from huggingface_hub import notebook_login

# Hugging Face 계정 로그인 및 토큰 설정
notebook_login()


In [10]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# 모델 및 프로세서 로드
model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-japanese"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# 음성 파일 로드 및 전처리
def load_audio(file_path):
    speech, sample_rate = torchaudio.load(file_path)
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(sample_rate, 16000)
        speech = resampler(speech)
    speech = torch.mean(speech, dim=0)  # 다중 채널을 단일 채널로 변환
    return speech




Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-japanese were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-japanese and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']

In [ ]:
# 음성 파일 경로
audio_file = "/home/eternal/song/output_directorys/04. Memories/vocals.wav"
speech = load_audio(audio_file)

# 모델 입력 준비
input_values = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True).input_values

# 음성 인식 수행
with torch.no_grad():
    logits = model(input_values).logits

# 결과 디코딩
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])

print("Transcription: ", transcription)